---
title: Scientific project utils
---

## Inspirations

- [DrWatson.jl](https://juliadynamics.github.io/DrWatson.jl/stable/)
- [sacred](https://sacred.readthedocs.io/en/stable/)


In [ ]:
# | default_exp project
# | export
import os
from beforerr.io import load, save
from datetime import timedelta
from pathlib import Path
from pydantic import validate_call

import warnings
from typing import Callable, Tuple, Union
from loguru import logger

In [ ]:
from fastcore.test import test_eq

## Naming Tools

In [ ]:
# | exporti
def valtostring(val, digits: Union[int, None], sigdigits: int) -> str:
    if isinstance(val, float):
        if digits is not None:
            return f"{val:.{digits}f}"
        elif sigdigits is not None:
            return f"{val:.{sigdigits}g}"
    return str(val)

In [ ]:
# | export
def savename(
    c: dict,
    prefix: str = "",
    suffix: str = "",
    allowedtypes: Tuple = (int, float, str, bool, timedelta),
    accesses: list[str] = [],
    ignores: list[str] = [],
    digits: int = None,
    connector: str = "_",
    equals: str = "=",
    expand: list[str] = [],
    sigdigits: int = 3,
    val2string: Callable = None,
    sort: bool = True,
) -> str:
    """
    Create a shorthand name based on the parameters in the dictionary `c`.

    Parameters
    ----------
    c : Dict
        The input dictionary containing the parameters.
    prefix : str, optional
        The prefix to start the name with.
    suffix : str, optional
        The suffix to end the name with.
    allowedtypes : Tuple, optional
        Only values of these types are included. (default: (int, float, str, bool))
    accesses : list[str], optional
        Specific keys to include in the name.
    ignores : list[str], optional
        Specific keys to ignore.
    digits : Union[int, None], optional
        Number of decimal digits for rounding floats.
    connector : str, optional
        String used to connect key-value pairs.
    equals : str, optional
        Connector between key and value.
    expand : list[str], optional
        Keys that will be expanded to their nested savename.
    sigdigits : int, optional
        Number of significant digits for rounding floats.
    val2string : Callable, optional
        Function to convert values to strings.
    sort : bool, optional
        Whether to sort the keys alphabetically.


    Returns
    -------
    str
        The generated shorthand name.
    """

    if any(sep in prefix for sep in ["/", "\\"]):
        warnings.warn(
            "Path separators in `savename` prefixes may break reproducibility on other OS. "
            "The recommended way is using `os.path.join` with `savename` (e.g. `os.path.join(datadir, savename(prefix, data))`)."
        )

    sigdigits = None if digits is not None else sigdigits
    val2string = val2string or (lambda val: valtostring(val, digits, sigdigits))

    labels = accesses or list(c.keys())
    p = (
        sorted(range(len(labels)), key=lambda k: labels[k])
        if sort
        else range(len(labels))
    )

    first = not prefix or prefix.endswith(os.path.sep)
    s = prefix
    for j in p:
        label = labels[j]
        if label in ignores:
            continue
        val = c[label]
        t = type(val)
        if any(issubclass(t, x) for x in allowedtypes):
            if label in expand:
                if not val:
                    continue
                sname = savename(
                    val,
                    connector=",",
                    digits=digits,
                    sigdigits=sigdigits,
                    equals=equals,
                )
                if not sname:
                    continue
                entry = f"{label}{equals}({sname})"
            else:
                entry = f"{label}{equals}{val2string(val)}"
            if not first:
                s += connector
            s += entry
            first = False

    if suffix:
        s += f".{suffix}"

    return s

In [ ]:
# Examples
d = {"a": 0.153456453, "b": 5, "mode": "double"}
test_eq(savename(d), "a=0.153_b=5_mode=double")
test_eq(savename(d, digits=4), "a=0.1535_b=5_mode=double")
test_eq(savename(d, prefix="n"), "n_a=0.153_b=5_mode=double")
test_eq(savename(d, suffix="n"), "a=0.153_b=5_mode=double.n")
test_eq(
    savename(d, prefix="n", suffix="n", connector="-"), "n-a=0.153-b=5-mode=double.n"
)
test_eq(savename(d, allowedtypes=(str,)), "mode=double")
test_eq(savename(d, connector=" | ", equals=" = "), "a = 0.153 | b = 5 | mode = double")

rick = {"never": "gonna", "give": "you", "up": "!"}
test_eq(savename(rick), "give=you_never=gonna_up=!")
test_eq(savename(rick, ignores=["up"]), "give=you_never=gonna")

In [ ]:
# | export
def append_prefix_suffix(name: str, prefix: str, suffix: str):
    if prefix:
        name = f"{prefix}_{name}"
    if suffix:
        name = f"{name}.{suffix}"
    return name

## Directory tools

In [ ]:
# | export
def projectdir():
    try:
        path = Path(os.getenv("PIXI_PROJECT_ROOT"))
    except TypeError:
        path = Path().cwd()
    return path


def datadir(subdir="data"):
    return projectdir() / subdir


def plotsdir(subdir="figures"):
    return projectdir() / subdir

In [ ]:
# | export
def setup_run_dir(
    c: dict, base_dir: Callable = datadir, change_dir: bool = True, **kwargs
):
    """
    Create a run directory based on the parameters in the dictionary `c`.

    Parameters
    ----------
    c
        The dictionary containing the parameters.
    base_dir
        The base directory to create the run directory in.

    Returns
    -------
    Path
        The run directory.
    """

    sub_dir = savename(c, **kwargs)
    base_dir = Path(base_dir()) if callable(base_dir) else Path(base_dir)
    directory = base_dir / sub_dir
    os.makedirs(directory, exist_ok=True)
    if change_dir:
        os.chdir(directory)
        logger.info(f"Changed directory to {directory}")

## Saving Tools

In [ ]:
# | exporti
@validate_call
def increment_backup_num(filepath: Path):
    if "_" in filepath.stem and filepath.stem.rsplit("_", 1)[-1].isdigit():
        base, num = filepath.stem.rsplit("_", 1)
        new_num = int(num) + 1
    else:
        base, new_num = filepath.stem, 1
    return filepath.with_name(f"{base}_{new_num}{filepath.suffix}")

In [ ]:
# | export
def safesave(file: str, data, save_func: Callable = save):
    path = Path(file)

    if path.exists():
        backup_path = increment_backup_num(path)
        while backup_path.exists():
            backup_path = increment_backup_num(backup_path)
        path.fs.mv(path, backup_path)
    return save_func(path, data)

### Produce or Load

[Produce or Load - DrWatson.jl](https://juliadynamics.github.io/DrWatson.jl/stable/save/#Produce-or-Load)


In [ ]:
# | export
def produce_or_load_file(
    f: Callable,
    config: dict,
    file: Path,
    force: bool = False,
    verbose: bool = True,
    load_func: Callable = load,
    save_func: Callable = save,
    **kwargs,
):
    exist = file.is_file()
    if not force and exist:
        data = load_func(file, **kwargs)
        return data, file
    else:
        if verbose:
            force and print(f"Producing file {file} now...")
            not exist and print(f"File {file} does not exist. Producing it now...")

        data = f(**config)

        try:
            save_func(file, data, **kwargs)
            verbose and print(f"File {file} saved.")
        except Exception as e:
            print(f"Could not save file. Error: {e}")

        return data, file


def produce_or_load(
    f: Callable,
    config: dict = dict(),
    path: Path = datadir(),
    suffix: str = "pickle",
    prefix: str = None,
    force: bool = False,
    verbose: bool = True,
    action_kwargs: dict = {},
    filename: Callable | str | None = None,
    **kwargs,
):
    if filename is None:
        name = savename(prefix, config, suffix, **kwargs)
    elif callable(filename):
        name = filename(config)
    elif isinstance(filename, str):
        name = filename
    else:
        raise ValueError("filename must be a callable or a string.")

    name = append_prefix_suffix(filename, prefix, suffix)

    file = path / name

    return produce_or_load_file(f, config, file, force, verbose, **action_kwargs)